# Examples from TTR overview lecture in `pyttr`

In [1]:
from ttrtypes import Type,BType,PType,Pred,Possibility,RecType,Fun,\
                     TTRStringType,KPlusStringType,LazyObj
from records import Rec
from utils import show

## Judgement

In [2]:
T = Type()
T.judge('a')

print(T.query('a'))

True


## Basic types

In [3]:
Ind = BType('Ind')
Ind.judge('a')

print(Ind.query('a'))

True


## Intensionality

In [4]:
T1 = Type()
T2 = Type()
T1.judge('a')
T1.judge('b')
T2.judge('a')
T2.judge('b')

print(T1.witness_cache)
print(T1.witness_cache == T2.witness_cache)
print(T1==T2)

['a', 'b']
True
False


## Ptypes

In [5]:
run = Pred('run',[Ind])
hug = Pred('hug',[Ind,Ind])
Ind.judge('b')
Ind.judge('d')
run_d = PType(run,['d'])
hug_b_d = PType(run,['b','d'])
run_d.judge('e1')
hug_b_d.judge('e2')

print(run_d.query('e1'))
print(hug_b_d.query('e1'))

True
False


## Possibilities (models)

In [6]:
M1 = Possibility('M1')

Ind.in_poss(M1)
run_d.in_poss(M1)
hug_b_d.in_poss(M1)

print(show(M1))


M1:
_____________________________________________
Ind: [a, b, d]
run(b, d): [e2]
run(d): [e1]
_____________________________________________



In [7]:
M2 = Possibility('M2')

BType('Ind').in_poss(M2).judge('c')
PType(run,['c']).in_poss(M2).judge('e1')

print(show(M2))


M2:
_____________________________________________
Ind: [c]
run(c): [e1]
_____________________________________________



## Record Types

In [8]:
boy = Pred('boy',[Ind])
dog = Pred('dog',[Ind])
a_boy_hugs_a_dog = RecType({'x':Ind,
                            'c_boy':(Fun('v',Ind,PType(boy,['v'])), ['x']),
                            'y':Ind,
                            'c_dog':(Fun('v',Ind,PType(dog,['v'])), ['y']),
                            'e':(Fun('v1',Ind,Fun('v2',Ind, PType(hug,['v1','v2']))), 
                                     ['x','y'])})
print(show(a_boy_hugs_a_dog))

{x : Ind, e : (lambda v1:Ind . lambda v2:Ind . hug(v1, v2), [x, y]), c_boy : (lambda v:Ind . boy(v), [x]), c_dog : (lambda v:Ind . dog(v), [y]), y : Ind}


In [9]:
M3 = Possibility('M3')
Ind3 = BType('Ind')
Ind3.in_poss(M3)
Ind3.judge('a')
Ind3.judge('b')
PType(boy,['a']).in_poss(M3).judge('s1')
PType(dog,['b']).in_poss(M3).judge('s2')
PType(hug,['a','b']).in_poss(M3).judge('s3')

print(show(M3))


M3:
_____________________________________________
Ind: [a, b]
dog(b): [s2]
hug(a, b): [s3]
boy(a): [s1]
_____________________________________________



In [10]:
r1 = Rec({'x':'a','c_boy':'s1','y':'b','c_dog':'s2','e':'s3'})

print(show(r1))

{x = a, e = s3, c_boy = s1, c_dog = s2, y = b}


In [11]:
print(a_boy_hugs_a_dog.in_poss(M3).query(r1))

True


## String types

May not yet work in general.

In [12]:
M4 = Possibility('M4')
pick_up = Pred('pick_up',[Ind,Ind])
attract_attention = Pred('attract_attention',[Ind,Ind])
throw = Pred('throw',[Ind,Ind])
run_after = Pred('run_after',[Ind,Ind])
return_stick = Pred('return_stick',[Ind,Ind,Ind])
Ind4 = BType('Ind')
Ind4.in_poss(M4)
Ind4.judge('a')
Ind4.judge('b')
Ind4.judge('c')
pick_up_a_c = PType(pick_up,['a','c'])
attract_attention_a_b = PType(attract_attention,['a','b'])
throw_a_c = PType(throw,['a','c'])
run_after_b_c = PType(run_after,['b','c'])
pick_up_b_c = PType(pick_up,['b','c'])
return_stick_b_c_a = PType(return_stick,['b','c','a'])
FetchOnce_a_b_c = TTRStringType([pick_up_a_c,attract_attention_a_b,throw_a_c,run_after_b_c,
                                 pick_up_b_c,return_stick_b_c_a])
Fetch_a_b_c = KPlusStringType(FetchOnce_a_b_c)

print(show(Fetch_a_b_c))


pick_up(a, c)^attract_attention(a, b)^throw(a, c)^run_after(b, c)^pick_up(b, c)^return_stick(b, c, a)+


In [13]:
human = Pred('human',[Ind])
stick = Pred('stick',[Ind])
play_fetch = Pred('play_fetch',[Ind,Ind])
T = RecType({'x':Ind,
             'c_human':(Fun('v',Ind,PType(human,['v'])),['x']),
             'y':Ind,
             'c_dog':(Fun('v',Ind,PType(dog,['v'])),['y']),
             'z':Ind,
             'c_stick':(Fun('v',Ind,PType(stick,['v'])),['z']),
             'e':(Fun('v1',Ind,Fun('v2',Ind,Fun('v3',Ind, 
                                                TTRStringType([PType(pick_up,['v1','v3']),
                                                               PType(attract_attention,['v1','v2'])])))),
                 ['x','y','z'])})
f = Fun('r',T,RecType({'e':PType(play_fetch,[LazyObj(['r','.','x']),LazyObj(['r','.','y'])])}))

print(show(f))

lambda r:{c_human : (lambda v:Ind . human(v), [x]), z : Ind, y : Ind, x : Ind, e : (lambda v1:Ind . lambda v2:Ind . lambda v3:Ind . pick_up(v1, v3)^attract_attention(v1, v2), [x, y, z]), c_dog : (lambda v:Ind . dog(v), [y]), c_stick : (lambda v:Ind . stick(v), [z])} . {e : play_fetch([r, ., x], [r, ., y])}
